In [9]:
!pip install pyspark

from google.colab import drive
drive.mount('/content/drive')

import pyspark.sql.functions as f
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, IntegerType, StringType, FloatType, StructField,BooleanType

schema = StructType([
    StructField("CrimeRate", FloatType(), True), 
    StructField("Youth", IntegerType(), True), 
    StructField("Southern", IntegerType(), True), 
    StructField("Education", FloatType(), True), 
    StructField("ExpenditureYear0", IntegerType(), True), 
    StructField("LabourForce", IntegerType(), True), 
    StructField("Males", IntegerType(), True), 
    StructField("MoreMales", IntegerType(), True), 
    StructField("StateSize", IntegerType(), True), 
    StructField("YouthUnemployment", IntegerType(), True), 
    StructField("MatureUnemployment", IntegerType(), True),
    StructField("HighYouthUnemploy", IntegerType(), True),
    StructField("Wage", IntegerType(), True),
    StructField("BelowWage", IntegerType(), True)
    ])

spark = SparkSession.builder.appName("Сrime").getOrCreate()
df = spark.read.format("csv").option("header", True).schema(schema).load("Crime.csv")
df.printSchema()
df.show()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
root
 |-- CrimeRate: float (nullable = true)
 |-- Youth: integer (nullable = true)
 |-- Southern: integer (nullable = true)
 |-- Education: float (nullable = true)
 |-- ExpenditureYear0: integer (nullable = true)
 |-- LabourForce: integer (nullable = true)
 |-- Males: integer (nullable = true)
 |-- MoreMales: integer (nullable = true)
 |-- StateSize: integer (nullable = true)
 |-- YouthUnemployment: integer (nullable = true)
 |-- MatureUnemployment: integer (nullable = true)
 |-- HighYouthUnemploy: integer (nullable = true)
 |-- Wage: integer (nullable = true)
 |-- BelowWage: integer (nullable = true)

+---------+-----+--------+---------+----------------+-----------+-----+---------+---------+-----------------+------------------+-----------------+----+---------+


In [13]:
print("Завдання 1")
print("Створити нову таблицю, яка буде містити наступні два стовпці: витрати на поліцію (розбити весь діапазон на десять частин); масив кількості чоловіків на 1000 нас, що відповідають цій частоті.")

from pyspark.sql.functions import array
from pyspark.sql.functions import col
from pyspark.sql.functions import split
from pyspark.sql.functions import lit
from pyspark.sql.functions import explode

df.createOrReplaceTempView("View")
df2 = spark.sql("SELECT ExpenditureYear0, collect_set(Males) AS Males FROM View GROUP BY ExpenditureYear0 ORDER BY ExpenditureYear0")
df2.show()

print("Завдання 2")
print("Взяти таблицю з завдання 1 і перетворити другий стовпець на кілька стовпців.")

df3 = df2.select("ExpenditureYear0", df2.Males[0], df2.Males[1], df2.Males[2]).show()

Завдання 1
Створити нову таблицю, яка буде містити наступні два стовпці: витрати на поліцію (розбити весь діапазон на десять частин); масив кількості чоловіків на 1000 нас, що відповідають цій частоті.
+----------------+---------------+
|ExpenditureYear0|          Males|
+----------------+---------------+
|              45|          [969]|
|              46|          [968]|
|              47|          [962]|
|              51|         [1024]|
|              55|         [1045]|
|              56|          [968]|
|              57|          [986]|
|              58|[974, 950, 973]|
|              61|          [953]|
|              62|          [986]|
|              63|     [984, 972]|
|              65|          [955]|
|              66|          [977]|
|              67|          [972]|
|              69|          [965]|
|              71|         [1029]|
|              72|          [998]|
|              74|          [984]|
|              75|     [996, 972]|
|              78|         [

In [15]:
print("Завдання 3")
print("Знайти середню частоту злочинів для усіх можливих поєднань параметрів Southern та HighYouthUnemploy.")

from pyspark.sql.functions import col, avg
import numpy as np
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType
from pyspark.sql.functions import expr


df.createOrReplaceTempView("View")
df4 = spark.sql("SELECT Southern, HighYouthUnemploy, collect_set(CrimeRate) AS CrimeRate FROM View GROUP BY Southern, HighYouthUnemploy ORDER BY Southern, HighYouthUnemploy")

query = """aggregate(
    `{col}`,
    CAST(0.0 AS double),
    (acc, x) -> acc + x,
    acc -> acc / size(`{col}`)
) AS  `avg_{col}`""".format(col="CrimeRate")

df4.selectExpr("*", query).show()


Завдання 3
Знайти середню частоту злочинів для усіх можливих поєднань параметрів Southern та HighYouthUnemploy.
+--------+-----------------+--------------------+------------------+
|Southern|HighYouthUnemploy|           CrimeRate|     avg_CrimeRate|
+--------+-----------------+--------------------+------------------+
|       0|                0|[127.2, 104.3, 11...|116.45294234331917|
|       0|                1|[67.6, 64.2, 115....| 88.67142731802804|
|       1|                0|[98.7, 96.8, 99.9...| 99.28666661580404|
|       1|                1|             [121.6]| 121.5999984741211|
+--------+-----------------+--------------------+------------------+



In [26]:
print("Завдання 4")
print("Додати в таблицю Crime новий стовпець, що містить різницю між") 
print("середнім та поточним значенням витрат на поліцію в залежності від")
print("того чи південний штат. Наприклад, якщо середні витрати в")
print("південних штатах 355, а в першому рядку витрати 300 і це південний") 
print("штат, то значення в додатковому стовпчику = 355-300=55. Зберегти") 
print("новий файл.")


import pyspark.sql.functions as f
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, IntegerType, StringType, FloatType, StructField,BooleanType

schema = StructType([
    StructField("CrimeRate", FloatType(), True), 
    StructField("Youth", IntegerType(), True), 
    StructField("Southern", IntegerType(), True), 
    StructField("Education", FloatType(), True), 
    StructField("ExpenditureYear0", IntegerType(), True), 
    StructField("LabourForce", IntegerType(), True), 
    StructField("Males", IntegerType(), True), 
    StructField("MoreMales", IntegerType(), True), 
    StructField("StateSize", IntegerType(), True), 
    StructField("YouthUnemployment", IntegerType(), True), 
    StructField("MatureUnemployment", IntegerType(), True),
    StructField("HighYouthUnemploy", IntegerType(), True),
    StructField("Wage", IntegerType(), True),
    StructField("BelowWage", IntegerType(), True)
    ])

spark = SparkSession.builder.appName("Сrime").getOrCreate()
df = spark.read.format("csv").option("header", True).schema(schema).load("Crime.csv")

df10 = spark.sql("SELECT avg(ExpenditureYear0) AS avg_ExpenditureYear0_Southern FROM View GROUP BY Southern ORDER BY Southern")
df10.show()

tab1 = df.filter(df.Southern == 0).withColumn("ExpenditureYear0_avg-ExpenditureYear0", (92 - df.ExpenditureYear0))
tab2 = df.filter(df.Southern == 1).withColumn("ExpenditureYear0_avg-ExpenditureYear0", (94 - df.ExpenditureYear0))
df9 = tab1.union(tab2)
df9.toPandas().to_csv("Crime2.csv")
df9.show(n=47)

Завдання 4
Додати в таблицю Crime новий стовпець, що містить різницю між
середнім та поточним значенням витрат на поліцію в залежності від
того чи південний штат. Наприклад, якщо середні витрати в
південних штатах 355, а в першому рядку витрати 300 і це південний
штат, то значення в додатковому стовпчику = 355-300=55. Зберегти
новий файл.
+-----------------------------+
|avg_ExpenditureYear0_Southern|
+-----------------------------+
|            92.87096774193549|
|                        69.75|
+-----------------------------+

+---------+-----+--------+---------+----------------+-----------+-----+---------+---------+-----------------+------------------+-----------------+----+---------+-------------------------------------+
|CrimeRate|Youth|Southern|Education|ExpenditureYear0|LabourForce|Males|MoreMales|StateSize|YouthUnemployment|MatureUnemployment|HighYouthUnemploy|Wage|BelowWage|ExpenditureYear0_avg-ExpenditureYear0|
+---------+-----+--------+---------+----------------+-----------+-